# 구조적 API 개요

- <strong>데이터 흐름을 정의</strong>하는 기본 추상화 개념
- <strong>비정형 로그 파일</strong>부터 <strong>반정형 CSV파일</strong>, <strong>매우 정형적인 파케이파일</strong>까지 다양한 유형의 데이터를 처리할 수 있다.
- 구조적 API에는 다음과 같은 세 가지 분산 컬렉션 API가 있음
  - DataSet
  - DataFrame
  - SQL 테이블과 뷰

## DataFrame과 Dataset
- 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션
- 모든 로우는 같은 데이터 타입 정보를 갖고 있어야함
- 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이며 불변성을 가짐

## 스키마
- 분산 컬렉션에 저장할 <strong>컬럼과 데이터 타입</strong>을 정의하는 방법
- 스키마는 데이터 소스에서 얻거나(schema-on-read) 직접 정의할 수 있음
- <strong>어떤 데이터 타입이 어느 위치에</strong> 있는지 정의하는 방법이 필요함

## 스파크의 구조적 데이터 타입 개요
- 스파크는 <strong>실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 갖고 있는 카탈리스트 엔진</strong>을 사용함
  - 카탈리스트 엔진은 다양한 <strong>실행 최적화</strong> 기능 제공
- 스파크는 자체 데이터 타입을 지원하는 <strong>여러 언어 API와 직접 매핑되며, 각 언어에 대한 매핑 테이블</strong>을 갖고 있음

In [0]:
df=spark.range(500).toDF('number')
df.select(df['number']+10)

Out[1]: DataFrame[(number + 10): bigint]

----
- 위 예제는 파이썬이 아닌 스파크 덧셈 연산을 수행
- 스파크가 지원하는 언어를 이용해 작성된 표현식을 <strong>카탈리스트 엔진에서 스파크의 데이터 타입으로 변환</strong>해 명령을 처리하기 때문

## DataFrame과 Dataset 비교
- 본질적으로 구조적 API에는 <strong>'비타입형'인 DataFrame과 '타입형'인 Dataset</strong>이 있음
  - DataFrame에도 데이터 타입이 있음
  - 하지만 스키마에 명시된 <strong>데이터 타입의 일치 여부를 런타임</strong>에 확인(DataFrame)하냐 / <strong>컴파일 타임</strong>에 확인(Dataset)하냐의 차이가 있음 
- 스파크 DataFrame은 <strong>Row 타입으로 구성된 DataSet</strong>임
  - Row 타입은 스파크가 사용하는 <strong>'연산에 최적화된 인메모리 포맷'</strong>의 내부적인 표현 방식임
  - Row 타입을 사용하면 가비지 컬렉션과 객체 초기화 부하가 있는 <strong>JVM 데이터 타입을 사용하는 대신 자체 데이터 포맷을 사용하므로 매우 효율적인 연산</strong>이 가능
  - <strong>어떤 언어 API를 사용하더라도 동일한 효과와 효율성</strong>을 얻을 수 있음
- 만약 컴파일 타임에 엄격한 데이터 검증이 필요하다면 DataSet 사용

## 컬럼
- 컬럼은 다음과 같은 값을 표현
  - 단순 데이터 타입(정수형, 문자열 등)
  - 복합 데이터 타입(배열, 맵 등)
  - null
- 스파크는 데이터 타입의 모든 정보를 추적하며 다양한 컬럼 변환 방법을 제공함

## 로우
- 로우는 데이터 레코드
- SQL, RDD, 데이터소스에서 얻거나 직접 만들 수 있음

In [0]:
#range 메서드를 사용해 Dataframe 생성 -> Row 객체로 이루어진 배열 반환
spark.range(2).collect()

Out[3]: [Row(id=0), Row(id=1)]

## 스파크 데이터 타입
- 스파크는 여러 내부 데이터 타입을 가지고 있음

In [0]:
from pyspark.sql.types import *

a= ByteType()
b= StringType()
c= BinaryType()

## 구조적 API의 실행 과정
1. DataFrame/ Dataset/ SQL을 이용해 코드를 작성한다.
2. 정상적인 코드라면 스파크가 논리적 실행 계획으로 변환한다.
3. 스파크는 논리적 실행 계획을 물리적 실행 계획으로 변환하며, 그 과정에서 추가적인 최적화를 할 수 있는지 확인한다.
4. 스파크는 클러스터에서 물리적 실행 계획(RDD 처리)을 실행한다.

### 논리적 실행 계획
1. 사용자 코드 -> 검증 전 논리적 실행 계획으로 변환
  - 사용자의 다양한 표현식을 <strong>최적화된 버전</strong>으로 변환함
  - <strong>코드의 유효성과 테이블이나 컬럼의 존재 여부만을 판단</strong>하는 과정
  - 아직 실행 계획을 검증하지 않은 상태<br><br>
2. 분석기
  - <strong>컬럼과 테이블을 검증</strong>하기 위해 카탈로그, 모든 테이블의 저장소, DataFrame의 정보 활용
  - 필요한 테이블이나 컬럼이 카탈로그에 없다면 검증 전 논리적 실행 계획이 만들어지지 않음<br><br>
3. 분석기에서의 검증 결과는 <strong>카탈리스트 옵티마이저</strong>로 전달 
  - 카탈리스트 옵티마이저는 <strong>논리적 실행 계획을 최적화</strong>하는 규칙의 모음<br><br>
4. <strong>최적화된 논리적 실행 계획</strong> 완성

### 물리적 실행 계획
- 스파크 실행 계획이라고도 불림
- 논리적 실행 계획을 클러스터 환경에서 실행하는 방법을 정의


1. <strong>다양한 물리적 실행 전략<strong> 생성
2. <strong>비용 모델</strong>을 이용하여 비교한 후 <strong>최적의 전략 선택<strong>

### 실행
- 물리적 실행 계획을 선정한 다음 저수준 프로그래밍 인터페이스인 RDD를 대상으로 모든 코드 실행
- 스파크는 런타임에 전체 task나 stage를 제거할 수 있는 자바 바이트 코드를 생성해 추가적인 최적화 수행